In [69]:
MODEL_PATH = "/Users/vineeshkumar/Desktop/model.tflite"
ZIP_PATH = "/Users/vineeshkumar/Desktop/inputimages.zip"    # your zip containing images
OUTPUT_XLSX = "/Users/vineeshkumar/Desktop/results.xlsx"


In [91]:
import numpy as np
import tensorflow as tf
from PIL import Image

interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

IMG_H = input_details[0]['shape'][1]
IMG_W = input_details[0]['shape'][2]

print("Model input:", IMG_H, "x", IMG_W, "RGB")


Model input: 28 x 28 RGB


In [92]:
import os
print(os.path.exists(ZIP_PATH))

True


In [115]:

extract_to = "/Users/vineeshkumar/Desktop/inputimages"

# Extract
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Show extracted file paths
for root, dirs, files in os.walk(extract_to):
    for file in files:
        //print(os.path.join(root, file))


SyntaxError: invalid syntax (2956391083.py, line 10)

In [114]:
import os
import random
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt

extract_dir = "inputimages"

image_paths = []

# Collect valid image files only
for root, dirs, files in os.walk(extract_dir):
    # Skip macOS hidden folder
    if "__MACOSX" in root:
        continue
    
    for f in files:
        # Skip macOS metadata files
        if f.startswith("._"):
            continue

        if f.lower().endswith((".png", ".jpg", ".jpeg")):
            image_paths.append(os.path.join(root, f))

# Pick random image
random_image_path = random.choice(image_paths)

# Display image safely
try:
    img = Image.open(random_image_path)
    plt.figure(figsize=(3,3))
    plt.imshow(img)
    plt.title(os.path.basename(random_image_path))
    plt.axis("off")
    plt.show()
    print("Displayed:", random_image_path)
except UnidentifiedImageError:
    print("Corrupted image:", random_image_path)




_IncompleteInputError: incomplete input (3518377435.py, line 39)

In [108]:
def preprocess_gray(img_path):
    print(img_path)
    img = Image.open(img_path).convert("L")  # GRAYSCALE
    img = img.resize((28, 28))
    arr = np.array(img).astype("float32") / 255.0

    # add channel dimension → (28,28) → (28,28,1)
    arr = np.expand_dims(arr, axis=-1)

    # add batch dimension → (1,28,28,1)
    arr = np.expand_dims(arr, axis=0)

    return arr, img.copy()



In [110]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

wb = Workbook()
ws = wb.active
ws.title = "Predictions"

ws.append(["Image", "Thumbnail", "Prediction", "Confidence"])

row = 2

for fname, img, pred, conf in results:

    ws.cell(row=row, column=1).value = fname
    ws.cell(row=row, column=3).value = pred
    ws.cell(row=row, column=4).value = conf

    # Save small thumbnail
    thumb_path = f"thumb_{row}.png"
    img.thumbnail((80, 80))
    img.save(thumb_path)

    xl_img = XLImage(thumb_path)
    ws.add_image(xl_img, f"B{row}")

    row += 1

wb.save(OUTPUT_XLSX)
print("results.xlsx generated successfully!")



results.xlsx generated successfully!
